In [ ]:
pip install dash


In [ ]:
!pip install pyngrok

In [ ]:
pip install ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 9.5 MB/s eta 0:00:00


In [ ]:
import dash
from dash import dcc, html  # Import dcc module
from pyngrok import ngrok
import pandas as pd
import subprocess
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from wordcloud import WordCloud
import io
import base64
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

authtoken = "2fK2j4SukN3nkgMe7yOhXlN0GAR_4BhHHYdyabo4JKt2BUPfU"

process = subprocess.Popen(['ngrok', 'authtoken', authtoken], stdout=subprocess.PIPE)
output, error = process.communicate()

if error:
    print("Error:", error)
else:
    print("Authtoken set successfully.")

# Load your DataFrame here (replace 'df' with your actual DataFrame)
df = pd.read_excel("DTI_Project_new.xlsx")

# Perform any necessary data processing here

# Check if 'TIMESTAMP' column exists in the DataFrame
if 'TIMESTAMP' not in df.columns:
    raise KeyError("Column 'TIMESTAMP' not found in DataFrame. Please check the column name.")

# Calculate country counts
country_counts = df['COUNTRY'].value_counts()
data = pd.DataFrame({'Country': country_counts.index, 'Count': country_counts.values})

# Create choropleth map
fig_choropleth = px.choropleth(data_frame=data, locations='Country', locationmode='country names',
                                color='Count', title='Number of Attacks Released By Country',
                                color_continuous_scale='Reds', range_color=(0, data['Count'].max()),
                                labels={'Count': 'Number of Attackers'})

# Plot frequency of each label
fig_label_frequency = {
    'data': [
        {'x': df['LABEL'].value_counts().index, 'y': df['LABEL'].value_counts().values, 'type': 'bar', 'marker': {'color': 'skyblue'}}
    ],
    'layout': {
        'title': 'Frequency of Each LABEL',
        'xaxis': {'title': 'LABEL'},
        'yaxis': {'title': 'Frequency'},
        'plot_bgcolor': '#f2f2f2',
        'paper_bgcolor': '#f2f2f2',
        'height': 500
    }
}

# Plot top countries with highest number of attackers (treemap)
top_countries = df['COUNTRY'].value_counts().head(10)
fig_treemap = px.treemap(names=top_countries.index, parents=["" for _ in top_countries.index], values=top_countries.values)
fig_treemap.update_layout(
    title='Top Countries with Highest Number of Attackers'
)

# Plot count of timestamps per country over time
if 'TIMESTAMP' in df.columns:
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    df['YEAR'] = df['TIMESTAMP'].dt.year
    timestamps_per_country_year = df.groupby(['COUNTRY', 'YEAR']).size().unstack(fill_value=0)

    fig_timestamps_per_country_plotly = go.Figure()
    for country in timestamps_per_country_year.index:
        fig_timestamps_per_country_plotly.add_trace(go.Scatter(x=timestamps_per_country_year.columns,
                                                                y=timestamps_per_country_year.loc[country],
                                                                mode='lines',
                                                                name=country))

    fig_timestamps_per_country_plotly.update_layout(title='Count of Timestamps per Country Over Time',
                                                    xaxis_title='Year',
                                                    yaxis_title='Count')
else:
    fig_timestamps_per_country_plotly = None

# Generate word cloud
titles = df['COUNTRY'].values
text = ' '.join(titles)
wordcloud = WordCloud(background_color='white', colormap='Blues').generate(text)

# Convert word cloud to image
buffer = io.BytesIO()
wordcloud.to_image().save(buffer, format='PNG')
encoded_image = base64.b64encode(buffer.getvalue()).decode('utf-8')

# Detect outliers using Isolation Forest and One-Class SVM
if 'TIMESTAMP' in df.columns:
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    time_window = pd.Timedelta(minutes=5)
    user_counts = df.groupby('USERNAME').apply(lambda group: group.set_index('TIMESTAMP').resample(time_window).size())

    anomalies = user_counts[user_counts > 5]

    anomalies_df = anomalies.reset_index()
    anomalies_df.columns = ['USERNAME', 'TIMESTAMP', 'OCCURRENCES']
    X = anomalies_df[['TIMESTAMP']]

    isolation_forest = IsolationForest()
    isolation_forest.fit(X)

    one_class_svm = OneClassSVM()
    one_class_svm.fit(X)

    # Predict outliers
    anomalies_df['IsolationForest_Outlier'] = isolation_forest.predict(X)
    anomalies_df['OneClassSVM_Outlier'] = one_class_svm.predict(X)

    # Outliers
    fig_isolation_forest_outliers = px.scatter(anomalies_df, x='TIMESTAMP', y='USERNAME', color='IsolationForest_Outlier',
                                               title='Isolation Forest Outliers', color_continuous_scale='viridis',
                                               labels={'TIMESTAMP': 'Timestamp', 'USERNAME': 'Username', 'IsolationForest_Outlier': 'Outlier'})

    fig_one_class_svm_outliers = px.scatter(anomalies_df, x='TIMESTAMP', y='USERNAME', color='OneClassSVM_Outlier',
                                            title='One-Class SVM Outliers', color_continuous_scale='viridis',
                                            labels={'TIMESTAMP': 'Timestamp', 'USERNAME': 'Username', 'OneClassSVM_Outlier': 'Outlier'})

else:
    fig_isolation_forest_outliers = None
    fig_one_class_svm_outliers = None

# Define layout
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Sentinel Scope Dashboard', style={'textAlign': 'center', 'fontFamily': 'Bookman Old Style'}),

    # Choropleth map
    dcc.Graph(
        id='choropleth-map',
        figure=fig_choropleth
    ),

    # Plot frequency of each label
    html.Div(children=[
        html.H2(children='Frequency of Each LABEL', style={'textAlign': 'center'}),
        dcc.Graph(
            id='label-frequency',
            figure=fig_label_frequency
        )
    ]),

    # Treemap for top countries with highest number of attackers
    dcc.Graph(
        id='top-countries-treemap',
        figure=fig_treemap
    ),

    # Plot count of timestamps per country over time
    dcc.Graph(
        id='timestamps-per-country',
        figure=fig_timestamps_per_country_plotly if fig_timestamps_per_country_plotly is not None else {}
    ),

    # Word cloud
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image),
                 style={'width': '80%', 'height': 'auto', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'})
    ], style={'textAlign': 'center'}),

    html.H2(children='ANOMALY DETECTION', style={'textAlign': 'center', 'fontFamily': 'Bookman Old Style'}),

    html.P("Since we don't have the GT (Ground Truth) values, we cannot get the accuracy. Hence, we are using unsupervised machine learning model, comparing with the help of graphs.",
           style={'textAlign': 'center', 'fontFamily': 'Bookman Old Style'}),

    # Plot Isolation Forest outliers
    dcc.Graph(
        id='isolation-forest-outliers',
        figure=fig_isolation_forest_outliers if fig_isolation_forest_outliers is not None else {}
    ),
    html.P("Yellow Dot - InLiner || Purple Dot - OutLiner - meaning unsual behavior", style={'textAlign': 'center', 'fontFamily': 'Bookman Old Style'}),
    # Plot One-Class SVM outliers
    dcc.Graph(
        id='one-class-svm-outliers',
        figure=fig_one_class_svm_outliers if fig_one_class_svm_outliers is not None else {}
    ),
    html.P("Yellow Dot - InLiner || Purple Dot - OutLiner - meaning unsual behavior", style={'textAlign': 'center', 'fontFamily': 'Bookman Old Style'}),
    # Chatbot section
    html.Div([
        html.Script(src="https://cdn.botpress.cloud/webchat/v1/inject.js"),
        html.Script("""
            window.botpressWebChat.init({
                "composerPlaceholder": "Chat with bot",
                "botConversationDescription": "This chatbot was built surprisingly fast with Botpress",
                "botId": "df61d435-bd06-4543-ac49-fc4db1383ec4",
                "hostUrl": "https://YOUR_BOTPRESS_INSTANCE_URL",  # Replace with your Botpress instance URL
                "messagingUrl": "https://messaging.botpress.cloud",
                "clientId": "df61d435-bd06-4543-ac49-fc4db1383ec4",
                "webhookId": "0ba47f0e-ad63-4f9c-9cff-ebdf5b4482af",
                "lazySocket": true,
                "themeName": "prism",
                "frontendVersion": "v1",
                "showBotInfoPage": true,
                "showPoweredBy": true,
                "theme": "prism",
                "themeColor": "#2563eb"
            });
        """)
    ])
])

if __name__ == '__main__':
    public_url = ngrok.connect(addr='8050')
    print("Dashboard URL:", public_url)

    app.run_server(debug=True)
